In [ ]:
"""
Created on Wed Jun 09 14:36 2021

Prepare bedrock slope for use in the neural network

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import nn_functions.data_formatting as dfmt


READ IN DATA

In [ ]:
nemo_run = 'OPM006'
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = dfmt.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
file_bed_corr = file_other_cut['corrected_isf_bathy']
file_bed_orig = file_mask_orig_cut['bathy_metry']
file_draft = file_other_cut['corrected_isfdraft'] 
file_isf_conc = file_conc_cut['isfdraft_conc']
file_bed_goodGL = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)


In [ ]:
bb_merid_slope = None
bb_zonal_slope = None

ice_merid_slope = None
ice_zonal_slope = None

dx = file_isf.x[2] - file_isf.x[1]
dy = file_isf.y[2] - file_isf.y[1]

isf_area_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()
entry_depth_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()

for kisf in tqdm(file_isf.Nisf):
    #print(kisf.values)
    bb_lonslope, bb_latslope = dfmt.slope_zonal_merid(kisf, file_isf, -1*file_bed_goodGL, dx, dy)
    #print('here1')
    ice_lonslope, ice_latslope = dfmt.slope_zonal_merid(kisf, file_isf, -1*file_draft, dx, dy)
    #print('here2')
    if bb_merid_slope is None:
        bb_merid_slope = bb_lonslope
        bb_zonal_slope = bb_latslope
        ice_merid_slope = ice_lonslope
        ice_zonal_slope = ice_latslope
    else:
        bb_merid_slope = bb_merid_slope.combine_first(bb_lonslope)
        bb_zonal_slope = bb_zonal_slope.combine_first(bb_latslope)
        ice_merid_slope = ice_merid_slope.combine_first(ice_lonslope)
        ice_zonal_slope = ice_zonal_slope.combine_first(ice_latslope)
     
    #print('here3')
    isf_area_map = isf_area_map.where(file_isf['ISF_mask']!=kisf, file_isf['isf_area_here'].sel(Nisf=kisf).drop('Nisf'))
    #print('here4')
    entry_depth_map = entry_depth_map.where(file_isf['ISF_mask']!=kisf, file_isf['front_bot_depth_max'].sel(Nisf=kisf).drop('Nisf'))
    #print('here5')
        
bb_merid_slope_smooth = bb_merid_slope.reindex_like(file_isf)
bb_zonal_slope_smooth = bb_zonal_slope.reindex_like(file_isf)
ice_merid_slope_smooth = ice_merid_slope.reindex_like(file_isf)
ice_zonal_slope_smooth = ice_zonal_slope.reindex_like(file_isf)


dataset_res_merid_bed = bb_merid_slope_smooth.to_dataset(name='slope_bed_lon')
dataset_res_zonal_bed = bb_zonal_slope_smooth.to_dataset(name='slope_bed_lat')
dataset_res_merid_ice = ice_merid_slope_smooth.to_dataset(name='slope_ice_lon')
dataset_res_zonal_ice = ice_zonal_slope_smooth.to_dataset(name='slope_ice_lat')


dataset_res = xr.merge([dataset_res_merid_bed,dataset_res_zonal_bed,dataset_res_merid_ice,dataset_res_zonal_ice,
                       isf_area_map.to_dataset(name='isf_area'),
                       entry_depth_map.to_dataset(name='entry_depth_max')])
dataset_res.to_netcdf(inputpath_mask+'nemo_5km_slope_info_bedrock_draft_latlon_oneFRIS.nc','w')